In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/'My Drive'/'All_Repos'/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines


In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import glob
%matplotlib inline

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1785856/45929032 bytes (3.9%)5677056/45929032 bytes (12.4%)9625600/45929032 bytes (21.0%)13418496/45929032 bytes (29.2%)17408000/45929032 bytes (37.9%)21446656/45929032 bytes (46.7%)25436160/45929032 bytes (55.4%)29458432/45929032 bytes (64.1%)33349632/45929032 bytes (72.6%)37289984/45929032 bytes (81.2%)41320448/45929032 bytes (90.0%)44793856/45929032 bytes (97.5%)

In [28]:


images = glob.glob('camera_cal/calibration*.jpg')
objpoints = []
imgpoints = []
  
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
for image in images:
  img = cv2.imread(image)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
  ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
  if ret == True:
    imgpoints.append(corners)
    objpoints.append(objp)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

class Line:
  
  def __init__(self):
    
    pass
  
    
def undistort_perspective_transform(img,mtx, dist,vertices,offset):

    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    img_size = (img.shape[1], img.shape[0])
    #print(img_size)
    leftupperpoint  = [568,470]
    rightupperpoint = [717,470]
    leftlowerpoint  = [260,680]
    rightlowerpoint = [1100,680]
    
    src = np.float32([leftupperpoint,leftlowerpoint,
                     rightupperpoint,rightlowerpoint])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result 
        # again, not exact, but close enough for our purposes
    dst = np.float32([[200,0], [200,680], [1100,0], [1100,680]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(undist, M, img_size)
    #plt.imshow(warped)
    #plt.show()
    return warped
    
  
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    #gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=sobel_kernel)
        scaled = np.abs(255*sobelx/np.max(sobelx)).astype('uint8')
    elif orient == 'y':
        sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=sobel_kernel)
        scaled = np.abs(255*sobely/np.max(sobely)).astype('uint8')
    grad_binary = np.zeros_like(scaled)
    grad_binary[(scaled > thresh[0]) & (scaled <= thresh[1])] = 1
    
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255),dir_thresh=(0.7,1.3)):
    # Calculate gradient magnitude
    # Apply threshold
    #gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    orient = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[((gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1]))] = 1

    return binary_output
  
def dir_threshold(image, sobel_kernel=3, dir_thresh=(0.66, 1.7)):
    # Calculate gradient direction
    # Apply threshold
    img = np.copy(image)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0,ksize=sobel_kernel) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobely = np.absolute(sobely)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    orient = np.arctan2(abs_sobely,abs_sobelx)
      
    # Threshold orient 
    
    orntbinary = np.zeros_like(scaled_sobel)
    orntbinary[(orient > dir_thresh[0]) & (orient < dir_thresh[1])] = 1
    
    return orient 
    
def pipeline(img, sobel_kernel=3,s_thresh=(90, 255), sx_thresh=(0, 255),sy_thresh=(20, 100),dir_thresh=(0,np.pi/2)):
    img = np.copy(img)
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    
    thresh = (100, 255)
    binary = np.zeros_like(R)
    binary[(R > thresh[0]) & (R <= thresh[1])] = 1
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold x gradient
    
    sxbinary = abs_sobel_thresh(s_binary, orient='x', sobel_kernel=sobel_kernel, thresh=sx_thresh)
    
    
    # Threshold orient 
    
    orntbinary = dir_threshold(img, sobel_kernel=sobel_kernel, dir_thresh=dir_thresh)
    
    
    
    
    
    combined = np.zeros_like(s_binary)
    #combined[((binary == 1) & (sxbinary == 1)) | ((s_binary == 1) & (orntbinary == 1))] = 1
    combined[(s_binary == 1)&(binary == 1)] = 1
    # Stack each channel
    #color_binary = np.dstack(( np.zeros_like(combined), combined, combined)) * 255
    #color_binary = np.dstack(( orntbinary, sxbinary, s_binary)) * 255
    return combined
  


def roi(img,vertices):
  
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img,img, mask)
    return masked_image

  
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Plots the left and right polynomials on the lane 
    #pts = np.array(pts, dtype=np.int32).reshape((-1, 1, 2))
    try:
      pts = []
      [pts.append([x,y]) for x,y in zip(left_fitx,ploty)]
      cv2.polylines(out_img, np.int32([pts]), 1, (255,255,0))
    except:
      pass
    
    # Plots the left and right polynomials on the lane 
    #print(len(pts))
    #pts = np.array(pts, dtype=np.int32).reshape((-1, 1, 2))
    try:
      pts = []
      [pts.append([x,y]) for x,y in zip(right_fitx,ploty)]
    #print(pts.shape,pts[:,0,:])
      cv2.polylines(out_img, np.int32([pts]), 1, (255,255,0))
    except:
      pass

    return out_img
  
def find_lane_lines(image):
  
  #grad_binary_x = abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(50, 255))
  #plt.imshow(grad_binary_x,cmap='gray')
  #plt.show()
  #mag_threshold = mag_thresh(image, sobel_kernel=9, mag_thresh=(30, 100),dir_thresh=(0.7,1.3))
  #print(image.shape)
  
  offset = 450
  
  vertices = np.array([[[200,680],[568,470],[717,470],[1100,680]]],dtype='int32')
  result = roi(image,vertices)
  result = undistort_perspective_transform(result,mtx, dist,vertices,offset)
  result = pipeline(result)
  #gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  #binary_warped = np.zeros_like(gray)
  #binary_warped[(gray > 0)] = 1
  result = fit_polynomial(result)
  return result
  #hist(image)
  
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
  #plt.imshow(image)
  #plt.show()
  
  #result = find_lane_lines(image)
  result = find_lane_lines(image)
  
  return result

  
def main():
  white_output = 'perspective.mp4'
  clip = VideoFileClip('/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines/project_video.mp4')
  edge_clip = clip.fl_image(process_image)
  #[process_image(frame) for frame in clip.iter_frames()]
  %time edge_clip.write_videofile(white_output, audio=False)
    
  
    
    #result = find_lane_pipeline(image)
    #return result

if __name__ == '__main__':
  main()

[MoviePy] >>>> Building video perspective.mp4
[MoviePy] Writing video perspective.mp4

















  0%|          | 0/1261 [00:00<?, ?it/s]














  0%|          | 1/1261 [00:00<03:23,  6.18it/s]














  0%|          | 2/1261 [00:00<03:33,  5.89it/s]














  0%|          | 3/1261 [00:00<03:33,  5.90it/s]














  0%|          | 4/1261 [00:00<03:31,  5.95it/s]














  0%|          | 5/1261 [00:00<03:28,  6.02it/s]














  0%|          | 6/1261 [00:01<03:26,  6.07it/s]














  1%|          | 7/1261 [00:01<03:24,  6.12it/s]














  1%|          | 8/1261 [00:01<03:25,  6.08it/s]














  1%|          | 9/1261 [00:01<03:24,  6.12it/s]














  1%|          | 10/1261 [00:01<03:21,  6.20it/s]














  1%|          | 11/1261 [00:01<03:22,  6.17it/s]














  1%|          | 12/1261 [00:01<03:21,  6.20it/s]














  1%|          | 13/1261 [00:02<03:20,  6.24it/s]














  1%|          | 14/1261 [00:02<03:20,  6.21it/s]














  1%|          | 15/1261 [00:02<03:19,  6.2

TypeError: ignored